In [ ]:
import json

In [ ]:
from index_mapping import load_mapping, matrix_index_to_root_id

mapping = load_mapping('root_id_to_index_mapping.json')
rootid_mapping = dict((v, k) for k, v in mapping.items())

In [5]:
import gzip

name_mapping = dict()

with gzip.open('./names.csv.gz', 'rt') as f:
    f.readline()
    for line in f:
        root_id, name, group = line.strip().split(',')
        root_id = int(root_id)
        name_mapping[root_id] = name

In [7]:
import numpy as np 

cluster_assignments = np.load('cluster_assignments.npy')

In [ ]:
# go through each cluster, 
# fine all root_id's within that cluster,
# convert each root_id to its name,
# and list them.
clusters = np.unique(cluster_assignments)

for cluster in clusters:
    print(f'Cluster {cluster}')
    cluster_indices = np.where(cluster_assignments == cluster)[0]
    cluster_root_ids = [matrix_index_to_root_id(i, mapping) for i in cluster_indices]
    cluster_names = [name_mapping[root_id] for root_id in cluster_root_ids]
    print(", ".join(cluster_names))

In [9]:
clusters

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18